# Knowledge Distillation for EfficientNetV2 on CIFAR-100 (SOTA) - Local Version

This notebook implements **State-of-the-Art** knowledge distillation with:

- **Decoupled Knowledge Distillation (DKD)** - Separates target and non-target class knowledge
- **CutMix + Mixup** - Advanced data augmentation

---

## Requirements

```bash
pip install torch torchvision tqdm thop matplotlib numpy
```

### Expected Results:

- **Teacher (EfficientNetV2-L):** ~68-70% accuracy
- **Distilled Student (DKD):** ~69-72% accuracy

---


In [1]:
# Cell 1: Setup and Imports
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import efficientnet_v2_s, efficientnet_v2_l, EfficientNet_V2_S_Weights, EfficientNet_V2_L_Weights
from tqdm import tqdm
import copy
import numpy as np
import matplotlib.pyplot as plt
import glob
import random
from pathlib import Path

# Setup Directories (Local paths)
PROJECT_ROOT = Path('./outputs')
MODEL_DIR = PROJECT_ROOT / 'models'
DATA_DIR = PROJECT_ROOT / 'data'
CHECKPOINT_DIR = PROJECT_ROOT / 'checkpoints'

MODEL_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Directories ready at: {PROJECT_ROOT.absolute()}")

# Reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.benchmark = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Directories ready at: d:\Projects\MasterProject\code\outputs
Using device: cuda
GPU: NVIDIA GeForce RTX 5070 Laptop GPU


In [2]:
# Cell 2: Experiment Configuration (SOTA)
# ==========================================
# HYPERPARAMETERS
# ==========================================
NUM_EPOCHS = 200
BATCH_SIZE = 128
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.05
PATIENCE = 30

# Distillation Params (DKD)
DKD_ALPHA = 1.0             # Weight for Target Knowledge
DKD_BETA = 8.0              # Weight for Non-Target Knowledge (Crucial)
TEMPERATURE = 4.0           # Softmax Temperature

# Augmentation Params
MIXUP_ALPHA = 0.8
CUTMIX_ALPHA = 1.0
CHECKPOINT_FREQUENCY = 20
NUM_CLASSES = 100

print(f"{'='*50}")
print(f"CONFIG: Epochs={NUM_EPOCHS} | Batch={BATCH_SIZE} | Temp={TEMPERATURE}")
print(f"DKD: Alpha={DKD_ALPHA}, Beta={DKD_BETA}")
print(f"{'='*50}")

CONFIG: Epochs=200 | Batch=128 | Temp=4.0
DKD: Alpha=1.0, Beta=8.0


In [3]:
# Cell 3: Data Loading
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

trainset = torchvision.datasets.CIFAR100(root=str(DATA_DIR), train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, 
                                          num_workers=4, pin_memory=True, drop_last=True)

testset = torchvision.datasets.CIFAR100(root=str(DATA_DIR), train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, 
                                         num_workers=4, pin_memory=True)

print(f"Data loaded: {len(trainset)} Training, {len(testset)} Test images")

Data loaded: 50000 Training, 10000 Test images


In [4]:
# Cell 4: Helper Functions (KD Loss, CutMix, Utils)

# ==========================================
# 1. Standard Knowledge Distillation Loss (Stable)
# Combines soft target loss with hard target loss
# ==========================================
def kd_loss(student_logits, teacher_logits, labels, temp=4.0, alpha=0.9):
    """
    Standard Knowledge Distillation Loss (Hinton et al.)
    - Soft loss: KL divergence between student and teacher soft targets
    - Hard loss: Cross-entropy with true labels
    - alpha: weight for soft loss (1-alpha for hard loss)
    """
    # Soft targets (KL Divergence)
    soft_student = F.log_softmax(student_logits / temp, dim=1)
    soft_teacher = F.softmax(teacher_logits / temp, dim=1)
    soft_loss = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (temp ** 2)
    
    # Hard targets (Cross Entropy)
    hard_loss = F.cross_entropy(student_logits, labels)
    
    # Combined loss
    loss = alpha * soft_loss + (1 - alpha) * hard_loss
    
    return loss

def kd_loss_mixup(student_logits, teacher_logits, labels_a, labels_b, lam, temp=4.0, alpha=0.9):
    """
    KD Loss for mixed samples (CutMix/Mixup compatible)
    """
    # Soft targets (KL Divergence) - same for both labels since teacher sees mixed input
    soft_student = F.log_softmax(student_logits / temp, dim=1)
    soft_teacher = F.softmax(teacher_logits / temp, dim=1)
    soft_loss = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (temp ** 2)
    
    # Hard targets (Mixed Cross Entropy)
    hard_loss = lam * F.cross_entropy(student_logits, labels_a) + \
                (1 - lam) * F.cross_entropy(student_logits, labels_b)
    
    # Combined loss
    loss = alpha * soft_loss + (1 - alpha) * hard_loss
    
    # NaN safety check
    if torch.isnan(loss):
        return hard_loss  # Fallback to CE only
    
    return loss

# ==========================================
# 2. Augmentations: Mixup & CutMix
# ==========================================
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def cutmix_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, bbx1:bbx2, bby1:bby2] = x[index, :, bbx1:bbx2, bby1:bby2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size()[-1] * x.size()[-2]))
    y_a, y_b = y, y[index]
    return x, y_a, y_b, lam

# ==========================================
# 3. Utilities (Save/Load/Evaluate)
# ==========================================
def evaluate_model_with_loss(model, dataloader, criterion):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    avg_loss = running_loss / len(dataloader)
    return acc, avg_loss

def save_checkpoint(model, optimizer, scheduler, epoch, best_acc, history, model_name, epochs_no_improve):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_acc': best_acc,
        'history': history,
        'epochs_no_improve': epochs_no_improve
    }
    path = CHECKPOINT_DIR / f"{model_name}_epoch{epoch+1}.pth"
    torch.save(checkpoint, path)
    print(f"  Checkpoint saved: {path}")
    
def load_checkpoint(model, optimizer, scheduler, model_name):
    checkpoints = sorted(glob.glob(str(CHECKPOINT_DIR / f"{model_name}_epoch*.pth")))
    if not checkpoints:
        return None
    latest = checkpoints[-1]
    print(f"  Loading checkpoint: {latest}")
    checkpoint = torch.load(latest, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return checkpoint

def cleanup_old_checkpoints(model_name, keep=3):
    checkpoints = sorted(glob.glob(str(CHECKPOINT_DIR / f"{model_name}_epoch*.pth")))
    if len(checkpoints) > keep:
        for chk in checkpoints[:-keep]:
            os.remove(chk)
            print(f"  Cleaned up: {os.path.basename(chk)}")

print("Helper functions loaded (Standard KD, CutMix, Mixup)")

Helper functions loaded (Standard KD, CutMix, Mixup)


In [5]:
# Cell 5: Optimized Training Loop
def train_model_optimized(model, dataloader, optimizer, scheduler, num_epochs, model_name, 
                         teacher_model=None, temp=4.0, kd_alpha=0.9,
                         patience=30, grad_clip=1.0):
    """
    Training loop with:
    - Standard KD loss (stable, proven)
    - CutMix/Mixup augmentation
    - Mixed precision training
    - Gradient clipping
    - NaN detection and recovery
    """
    
    # 1. Load Checkpoint
    checkpoint = load_checkpoint(model, optimizer, scheduler, model_name)
    if checkpoint:
        start_epoch = checkpoint['epoch'] + 1
        best_acc = checkpoint['best_acc']
        history = checkpoint['history']
        epochs_no_improve = checkpoint['epochs_no_improve']
        best_model_wts = copy.deepcopy(model.state_dict())
        print(f"  Resuming from epoch {start_epoch}, Best Acc: {best_acc:.2f}%")
    else:
        start_epoch = 0
        best_acc = 0.0
        history = {'train_loss': [], 'val_loss': [], 'val_accuracy': []}
        epochs_no_improve = 0
        best_model_wts = copy.deepcopy(model.state_dict())
        print(f"  Starting fresh training...")

    # 2. Setup
    scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())
    val_criterion = nn.CrossEntropyLoss()
    
    if teacher_model:
        teacher_model.eval()
        for param in teacher_model.parameters():
            param.requires_grad = False

    # 3. Training Loop
    for epoch in range(start_epoch, num_epochs):
        model.train()
        running_loss = 0.0
        valid_batches = 0
        
        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for inputs, labels in loop:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Randomly choose Mixup (50%) or CutMix (50%)
            use_cutmix = np.random.rand() > 0.5
            if use_cutmix:
                inputs_aug, labels_a, labels_b, lam = cutmix_data(inputs.clone(), labels, alpha=CUTMIX_ALPHA)
            else:
                inputs_aug, labels_a, labels_b, lam = mixup_data(inputs, labels, alpha=MIXUP_ALPHA)
            
            optimizer.zero_grad()
            
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                # Student Forward
                student_outputs = model(inputs_aug)
                
                # Loss Calculation
                if teacher_model:
                    # Teacher Forward (no grad)
                    with torch.no_grad():
                        teacher_outputs = teacher_model(inputs_aug)
                    
                    # Standard KD Loss with Mixup support
                    loss = kd_loss_mixup(
                        student_outputs, teacher_outputs, 
                        labels_a, labels_b, lam,
                        temp=temp, alpha=kd_alpha
                    )
                else:
                    # Standard CE for Teacher training
                    loss = lam * F.cross_entropy(student_outputs, labels_a) + \
                           (1 - lam) * F.cross_entropy(student_outputs, labels_b)
            
            # Skip batch if loss is NaN
            if torch.isnan(loss):
                loop.set_postfix(loss="NaN-skip")
                continue
            
            # Backward
            scaler.scale(loss).backward()
            
            if grad_clip > 0:
                scaler.unscale_(optimizer)
                nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                
            scaler.step(optimizer)
            scaler.update()
            
            running_loss += loss.item()
            valid_batches += 1
            loop.set_postfix(loss=f"{loss.item():.3f}")

        # Step Scheduler
        scheduler.step()
        
        # Validation
        train_loss = running_loss / max(valid_batches, 1)
        val_acc, val_loss = evaluate_model_with_loss(model, testloader, val_criterion)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_acc)
        
        current_lr = scheduler.get_last_lr()[0]
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | LR: {current_lr:.6f}")
        
        # Save Best
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), MODEL_DIR / f"{model_name}.pth")
            epochs_no_improve = 0
            print(f"  New best model saved! Accuracy: {best_acc:.2f}%")
        else:
            epochs_no_improve += 1
            
        # Checkpointing
        if (epoch + 1) % CHECKPOINT_FREQUENCY == 0:
            save_checkpoint(model, optimizer, scheduler, epoch, best_acc, history, model_name, epochs_no_improve)
            cleanup_old_checkpoints(model_name)
            
        # Early Stopping
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
    print(f"\nTraining complete. Best accuracy: {best_acc:.2f}%")
    model.load_state_dict(best_model_wts)
    return model, history

print("Training function loaded (with NaN protection)")

Training function loaded (with NaN protection)


In [6]:
# Cell 6: Initialize Models
print("Loading Teacher (EfficientNetV2-L)...")
teacher_model = efficientnet_v2_l(weights=EfficientNet_V2_L_Weights.IMAGENET1K_V1)
teacher_model.classifier[1] = nn.Linear(teacher_model.classifier[1].in_features, NUM_CLASSES)
teacher_model = teacher_model.to(device)

print("Loading Student (EfficientNetV2-S)...")
student_model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
student_model.classifier[1] = nn.Linear(student_model.classifier[1].in_features, NUM_CLASSES)
student_model = student_model.to(device)

print("Models loaded")

Loading Teacher (EfficientNetV2-L)...
Loading Student (EfficientNetV2-S)...
Models loaded


In [7]:
# Cell 7: Train/Load Teacher Model
print("\n" + "="*70)
print("TEACHER MODEL")
print("="*70)

teacher_path = MODEL_DIR / "teacher_model.pth"
if teacher_path.exists():
    print(f"Found existing Teacher Model: {teacher_path}")
    teacher_model.load_state_dict(torch.load(teacher_path, map_location=device))
else:
    print("Training Teacher Model (This may take a while)...")
    opt_t = optim.AdamW(teacher_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    sch_t = optim.lr_scheduler.CosineAnnealingLR(opt_t, T_max=NUM_EPOCHS)
    teacher_model, teacher_history = train_model_optimized(
        teacher_model, trainloader, opt_t, sch_t, NUM_EPOCHS, "teacher_model", teacher_model=None
    )

# Evaluate Teacher
teacher_model.eval()
teacher_accuracy, _ = evaluate_model_with_loss(teacher_model, testloader, nn.CrossEntropyLoss())
print(f"\nTeacher Accuracy: {teacher_accuracy:.2f}%")


TEACHER MODEL
Found existing Teacher Model: outputs\models\teacher_model.pth

Teacher Accuracy: 75.75%


In [8]:
# Cell 8: Train Distilled Student (Standard KD + CutMix)
print("\n" + "="*70)
print("DISTILLED STUDENT MODEL (Standard KD + CutMix/Mixup)")
print("="*70)

# KD Parameters
KD_ALPHA = 0.9  # Weight for soft loss (0.9 = 90% soft, 10% hard)
KD_TEMP = 4.0   # Temperature for softening

student_name = "distilled_student_kd"
student_path = MODEL_DIR / f"{student_name}.pth"

if student_path.exists():
    print(f"Found existing Distilled Model: {student_path}")
    student_model.load_state_dict(torch.load(student_path, map_location=device))
    distilled_accuracy, _ = evaluate_model_with_loss(student_model, testloader, nn.CrossEntropyLoss())
    print(f"Distilled Student Accuracy: {distilled_accuracy:.2f}%")
else:
    print(f"\nStarting Knowledge Distillation...")
    print(f"  KD Alpha (soft weight): {KD_ALPHA}")
    print(f"  Temperature: {KD_TEMP}")
    print(f"  Augmentation: CutMix + Mixup")
    
    # Re-initialize student model (fresh weights from ImageNet)
    student_model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
    student_model.classifier[1] = nn.Linear(student_model.classifier[1].in_features, NUM_CLASSES)
    student_model = student_model.to(device)
    
    # Optimizer & Scheduler
    opt_s = optim.AdamW(student_model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    sch_s = optim.lr_scheduler.CosineAnnealingLR(opt_s, T_max=NUM_EPOCHS)
    
    # Run Training
    trained_student, distilled_history = train_model_optimized(
        model=student_model,
        dataloader=trainloader,
        optimizer=opt_s,
        scheduler=sch_s,
        num_epochs=NUM_EPOCHS,
        model_name=student_name,
        teacher_model=teacher_model,
        temp=KD_TEMP,
        kd_alpha=KD_ALPHA,
        patience=PATIENCE
    )
    
    distilled_accuracy, _ = evaluate_model_with_loss(trained_student, testloader, nn.CrossEntropyLoss())
    print(f"\nDistilled Student Final Accuracy: {distilled_accuracy:.2f}%")


DISTILLED STUDENT MODEL (Standard KD + CutMix/Mixup)

Starting Knowledge Distillation...
  KD Alpha (soft weight): 0.9
  Temperature: 4.0
  Augmentation: CutMix + Mixup
  Starting fresh training...


Epoch 1/200: 100%|██████████| 390/390 [01:11<00:00,  5.44it/s, loss=1.068]


Epoch 1/200 | Train Loss: 1.4984 | Val Loss: 2.9059 | Val Acc: 30.31% | LR: 0.001000
  New best model saved! Accuracy: 30.31%


Epoch 2/200: 100%|██████████| 390/390 [01:16<00:00,  5.10it/s, loss=1.199]


Epoch 2/200 | Train Loss: 1.3106 | Val Loss: 2.6329 | Val Acc: 36.79% | LR: 0.001000
  New best model saved! Accuracy: 36.79%


...

Epoch 198/200 | Train Loss: 0.5981 | Val Loss: 1.2802 | Val Acc: 71.84% | LR: 0.000000


Epoch 199/200: 100%|██████████| 390/390 [01:10<00:00,  5.54it/s, loss=0.363]


Epoch 199/200 | Train Loss: 0.5968 | Val Loss: 1.2367 | Val Acc: 72.36% | LR: 0.000000
  New best model saved! Accuracy: 72.36%


Epoch 200/200: 100%|██████████| 390/390 [01:10<00:00,  5.54it/s, loss=0.498]


Epoch 200/200 | Train Loss: 0.5887 | Val Loss: 1.2385 | Val Acc: 72.19% | LR: 0.000000
  Checkpoint saved: outputs\checkpoints\distilled_student_kd_epoch200.pth
  Cleaned up: distilled_student_kd_epoch200.pth

Training complete. Best accuracy: 72.36%

Distilled Student Final Accuracy: 72.36%


In [9]:
# Cell 9: Results Summary
print("\n" + "="*80)
print("FINAL RESULTS SUMMARY")
print("="*80)

print("\n| Model                    | Accuracy (%) |")
print("|--------------------------|--------------|")
try:
    print(f"| Teacher (EfficientNet-L) | {teacher_accuracy:12.2f} |")
except NameError:
    print(f"| Teacher (EfficientNet-L) | {'N/A':>12} |")

try:
    print(f"| Distilled (DKD+CutMix)   | {distilled_accuracy:12.2f} |")
except NameError:
    print(f"| Distilled (DKD+CutMix)   | {'N/A':>12} |")

try:
    improvement = distilled_accuracy - teacher_accuracy
    print(f"\n{'='*80}")
    if improvement > 0:
        print(f"Student SURPASSED Teacher by: {improvement:+.2f}%")
    else:
        print(f"Gap from Teacher: {improvement:.2f}%")
    print(f"{'='*80}")
except NameError:
    pass

print(f"\nAll models saved to: {MODEL_DIR.absolute()}")
print(f"Checkpoints saved to: {CHECKPOINT_DIR.absolute()}")


FINAL RESULTS SUMMARY

| Model                    | Accuracy (%) |
|--------------------------|--------------|
| Teacher (EfficientNet-L) |        75.75 |
| Distilled (DKD+CutMix)   |        72.36 |

Gap from Teacher: -3.39%

All models saved to: d:\Projects\MasterProject\code\outputs\models
Checkpoints saved to: d:\Projects\MasterProject\code\outputs\checkpoints
